In [27]:
# Step 1: Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils 
import cv2


In [12]:
# Step 2: Import dataset
data_path = './Data/CNN_datasets/new'

# Use ImageDataGenerator for loading and augmenting the data
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split
)

# Create train and validation generators
train_generator = datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set to 'training' for the train split
)

val_generator = datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set to 'validation' for the validation split
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Display the class labels
print("Class labels:", class_labels)

# Optional: Convert class indices to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_generator.labels, num_classes=len(class_labels))
val_labels = tf.keras.utils.to_categorical(val_generator.labels, num_classes=len(class_labels))

# Optional: Inspect the first batch of images and labels
sample_images, sample_labels = next(train_generator)



Found 1204 images belonging to 4 classes.
Found 300 images belonging to 4 classes.
Class labels: ['buffalo', 'elephant', 'rhino', 'zebra']


In [13]:
# Step 7: Load Faster R-CNN model with MobileNetV2 backbone
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

# Create Faster R-CNN model
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(class_labels), activation='softmax')  # Adjust output classes based on your dataset
])

In [14]:
# Step 8: Train the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Assuming you have already prepared train_generator and val_generator
history = model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10
38/38 [==============================] - 56s 1s/step - loss: 0.5946 - accuracy: 0.7857 - val_loss: 0.2432 - val_accuracy: 0.9200
Epoch 2/10
38/38 [==============================] - 41s 1s/step - loss: 0.2046 - accuracy: 0.9352 - val_loss: 0.1667 - val_accuracy: 0.9433
Epoch 3/10
38/38 [==============================] - 41s 1s/step - loss: 0.1620 - accuracy: 0.9510 - val_loss: 0.1837 - val_accuracy: 0.9433
Epoch 4/10
38/38 [==============================] - 41s 1s/step - loss: 0.1339 - accuracy: 0.9568 - val_loss: 0.1170 - val_accuracy: 0.9667
Epoch 5/10
38/38 [==============================] - 40s 1s/step - loss: 0.1131 - accuracy: 0.9693 - val_loss: 0.1170 - val_accuracy: 0.9767
Epoch 6/10
38/38 [==============================] - 40s 1s/step - loss: 0.0928 - accuracy: 0.9743 - val_loss: 0.0959 - val_accuracy: 0.9633
Epoch 7/10
38/38 [==============================] - 40s 1s/step - loss: 0.1006 - accuracy: 0.9684 - val_loss: 0.1191 - val_accuracy: 0.9600
Epoch 8/10
38/38 [==

In [15]:
# Step 9: Evaluate using metrics
# Assuming you have already prepared test_generator
test_loss, test_accuracy = model.evaluate(val_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

10/10 [==============================] - 7s 630ms/step - loss: 0.1167 - accuracy: 0.9567
Test Loss: 0.11668369174003601, Test Accuracy: 0.9566666483879089


In [16]:
# Step 10: Save the model
model.save('faster_rcnn_gg1.h5')

In [21]:
# Step 11: Load the model
model = tf.keras.models.load_model('faster_rcnn_gg1.h5')

# Step 12: Make predictions
image_path = './Data/CNN_datasets/final_testing/1.jpeg'
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)
print(predictions)
print(class_labels[np.argmax(predictions[0])])



[[0.03886882 0.41592526 0.01804311 0.5271628 ]]
zebra


In [28]:
#Visually present the results

# Load the model
model = tf.keras.models.load_model('faster_rcnn_gg1.h5')

# Load the image
image_path = './Data/CNN_datasets/final_testing/1.jpeg'
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.

# Make predictions
predictions = model.predict(input_arr)
class_label = class_labels[np.argmax(predictions[0])]

# Convert image to NumPy array
image_np = cv2.imread(image_path)

# Get bounding box coordinates
ymin, xmin, ymax, xmax = predictions[0]

# Convert normalized coordinates to pixel values
height, width, _ = image_np.shape
xmin = int(xmin * width)
xmax = int(xmax * width)
ymin = int(ymin * height)
ymax = int(ymax * height)

# Draw bounding box on the image
color = (0, 255, 0)  # Green color for the bounding box
thickness = 2
cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), color, thickness)

# Display the image with bounding box
cv2.putText(image_np, class_label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
cv2.imshow('Object Detection', image_np)
cv2.waitKey(0)
cv2.destroyAllWindows()
